<a href="https://colab.research.google.com/github/vengaladasvivek140-sudo/AIAC/blob/main/AIAC_PROJECT_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display, HTML
from datetime import datetime
import sys

# -----------------------------------------------------------------
# Step 1 & 2: Load and Preprocess Data (Same as before)
# -----------------------------------------------------------------
print("Attempting to load 'city_day.csv'...")

try:
    df = pd.read_csv('city_day.csv')
    print("File loaded successfully.")
except FileNotFoundError:
    print("="*50)
    print("FATAL ERROR: 'city_day.csv' not found. Please upload the file.")
    print("="*50)
    sys.exit()
except Exception as e:
    print(f"An unexpected error occurred loading the file: {e}")
    sys.exit()

# --- Data Cleaning ---
if 'Datetime' in df.columns:
    df['Date'] = pd.to_datetime(df['Datetime'])
    print("Found 'Datetime' column and converted to date.")
elif 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    print("Found 'Date' column and converted to date.")
else:
    print("FATAL ERROR: Neither 'Date' nor 'Datetime' column found in the file.")
    sys.exit()

process_cols = ['PM2.5', 'PM10', 'NO2', 'AQI', 'AQI_Bucket']
missing_cols = [col for col in process_cols if col not in df.columns]
if missing_cols:
    print(f"FATAL ERROR: The following required columns are missing: {missing_cols}")
    sys.exit()

print("All required columns found.")
df.sort_values(by=['City', 'Date'], inplace=True)

print("Handling missing values...")
df['AQI_Bucket'] = df.groupby('City')['AQI_Bucket'].ffill()
df['AQI_Bucket'] = df.groupby('City')['AQI_Bucket'].bfill()
df[process_cols] = df.groupby('City')[process_cols].ffill()
df[process_cols] = df.groupby('City')[process_cols].bfill()
df.dropna(subset=process_cols, inplace=True)

city_list = df['City'].unique()
city_list.sort()
min_date = df['Date'].min().date()
max_date = df['Date'].max().date()

print(f"Data processed. Found {len(city_list)} cities: {city_list}")


# -----------------------------------------------------------------
# HINT: Helper function for Color-Coded Alerts (Same as before)
# -----------------------------------------------------------------
def get_aqi_alert_html(city, aqi_val, bucket, date_str):
    color_map = {
        'Good': '#79C479', 'Satisfactory': '#B2D17A', 'Moderate': '#F7E35E',
        'Poor': '#F49B5B', 'Very Poor': '#EC5E5E', 'Severe': '#B04A4A',
        'default': '#A0A0A0'
    }
    color = color_map.get(bucket, 'default')
    html = f"""
    <div style="
        background-color: {color}; border: 1px solid #333; border-radius: 5px;
        padding: 10px; margin: 5px; width: 200px; display: inline-block;
        color: {'black' if bucket in ['Good', 'Satisfactory', 'Moderate'] else 'white'};
    ">
        <strong style="font-size: 1.2em;">{city}</strong><br>
        <span style="font-size: 1.5em;">{aqi_val:.0f}</span>
        <span style="font-size: 0.9em;">({bucket})</span><br>
        <em style="font-size: 0.8em;">as of {date_str}</em>
    </div>
    """
    return html

# -----------------------------------------------------------------
# Step 4 & 5: Build a STABLE Dashboard (ROCK-SOLID "Output" Method)
# -----------------------------------------------------------------

# 1. Create the widgets (CONTROLS)
city_selector = widgets.SelectMultiple(
    options=city_list,
    value=['Delhi', 'Mumbai', 'Bangalore'],
    description='Select Cities:',
    disabled=False,
    style={'description_width': 'initial'},
)

start_date_picker = widgets.DatePicker(
    description='Start Date',
    value=datetime(2024, 1, 1).date(),
    style={'description_width': 'initial'}
)

end_date_picker = widgets.DatePicker(
    description='End Date',
    value=max_date,
    style={'description_width': 'initial'}
)

# 2. Create the output areas
alert_output = widgets.Output()
# --- NEW ---
# Create simple Output() widgets for the plots
pollutant_plot_output = widgets.Output()
aqi_plot_output = widgets.Output()
# -----------


# 3. Define the main function to update plots and alerts
def on_widget_change(change):

    # Clear all outputs
    alert_output.clear_output(wait=True)
    pollutant_plot_output.clear_output(wait=True)
    aqi_plot_output.clear_output(wait=True)

    selected_cities = city_selector.value
    start_date = start_date_picker.value
    end_date = end_date_picker.value

    # --- Validation ---
    if not selected_cities or not start_date or not end_date:
        with alert_output:
            display(HTML("<h3 style='color:red;'>Please select at least one city and a valid date range.</h3>"))
        return
    if start_date > end_date:
        with alert_output:
            display(HTML("<h3 style='color:red;'>Error: Start date must be before end date.</h3>"))
        return
    # --- End Validation ---

    filtered_df = df[
        (df['City'].isin(selected_cities)) &
        (df['Date'].dt.date >= start_date) &
        (df['Date'].dt.date <= end_date)
    ]

    if filtered_df.empty:
        with alert_output:
            display(HTML("<h3 style='color:orange;'>No data available for the selected cities and date range.</h3>"))
        return

    # --- HINT: Display Color-Coded Alerts ---
    with alert_output:
        html_boxes = ""
        for city in selected_cities:
            city_data = filtered_df[filtered_df['City'] == city]
            if not city_data.empty:
                latest_data = city_data.iloc[-1]
                html_boxes += get_aqi_alert_html(
                    city=city, aqi_val=latest_data['AQI'],
                    bucket=latest_data['AQI_Bucket'],
                    date_str=latest_data['Date'].strftime('%Y-%m-%d')
                )
        display(HTML(f"<h2>Latest AQI Status</h2>{html_boxes}"))

    # --- Step 3: Visualize Trends (Render in the Output areas) ---

    # --- PLOT 1: Pollutants ---
    with pollutant_plot_output:
        pollutant_df = filtered_df.melt(
            id_vars=['Date', 'City'],
            value_vars=['PM2.5', 'PM10', 'NO2'], # Only pollutants
            var_name='Pollutant',
            value_name='Concentration (μg/m³)'
        )
        fig_pollutants = px.line(
            pollutant_df, x='Date', y='Concentration (μg/m³)',
            color='Pollutant', facet_col='City',
            facet_col_wrap=2, title='Pollutant Trends (PM2.5, PM10, NO2)'
        )
        fig_pollutants.update_layout(height=500)
        display(fig_pollutants)

    # --- PLOT 2: AQI ---
    with aqi_plot_output:
        aqi_df = filtered_df.melt(
            id_vars=['Date', 'City'],
            value_vars=['AQI'], # Only AQI
            var_name='Index',
            value_name='AQI Value'
        )
        fig_aqi = px.line(
            aqi_df, x='Date', y='AQI Value',
            color='City', # Color by city
            title='Air Quality Index (AQI) Trend'
        )
        fig_aqi.update_layout(height=400)
        display(fig_aqi)
    # -----------------------------------------------------


# 4. Link widgets to the update function
city_selector.observe(on_widget_change, names='value')
start_date_picker.observe(on_widget_change, names='value')
end_date_picker.observe(on_widget_change, names='value')

# 5. Display the final dashboard (using a simple VBox)
print("\n--- Interactive Air Pollution Dashboard ---")

# --- Create the Layout (SIMPLE VBox) ---
header_html = """
<div style="
    background-color: #2E4053; /* A simple, dark blue */
    color: white;
    padding: 12px;
    border-radius: 8px;
    text-align: center;
">
    <h1 style="margin: 0; font-size: 1.8em;">🇮🇳 Indian Cities Air Pollution Dashboard</h1>
</div>
"""
header = widgets.HTML(header_html)

# Put controls in a horizontal box
controls = widgets.HBox(
    [city_selector, start_date_picker, end_date_picker]
)
controls.layout.margin = '15px 0 15px 0'

# Stack everything vertically: Header, Controls, Alerts, Graphs
app = widgets.VBox([
    header,
    controls,
    alert_output,
    pollutant_plot_output,
    aqi_plot_output
])
# -------------------------

# Display the final app
display(app)

# 6. Trigger the first run
on_widget_change(None)

Attempting to load 'city_day.csv'...
File loaded successfully.
Found 'Datetime' column and converted to date.
All required columns found.
Handling missing values...
Data processed. Found 5 cities: ['Bangalore' 'Chennai' 'Delhi' 'Kolkata' 'Mumbai']

--- Interactive Air Pollution Dashboard ---


In [10]:
from google.colab import output
output.enable_custom_widget_manager()

In [6]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [7]:
from google.colab import output
output.disable_custom_widget_manager()